### PCAM Eksperymenty - Seria 1

### Wspólne

In [1]:
# Tego nie odpalać -- to przeładuje moduły wszystkie i potencjalnie zresetuje LazyLoadera
# Nie ruszać chyba, że wiecie co robicie

# %load_ext autoreload
# %autoreload 0

In [1]:
import sys
sys.path.append("../..")

In [2]:
import torch
# import torch.nn as nn
import torch.nn.functional as F
# from torch.utils.data import Subset, Dataset, DataLoader
# from torcheval.metrics.functional import multiclass_f1_score, multiclass_accuracy
import torchvision.transforms as T

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import src.modules.data_module as dm
import src.modules.learner_module as lm
import src.modules.evaluation_module as em
import src.modules.pipeline as pp

import src.models as models

dm.update_data_dir('../../data')
device = (
    "cuda" if torch.cuda.is_available() else "cpu"
)
print(f"Using {device} device")

Initialising Lazy Loader (PCAMLL)
Default data directory set to c:\MyRoot\PWFiles\semestr 6\WB\wb-active-learning\notebooks\common\data
To change this path, use the update_data_dir() function from the data_module
Data directory set to: c:\MyRoot\PWFiles\semestr 6\WB\wb-active-learning\data
Using cuda device


### Jasiek
- `N`: 5
- `init`: 500
- `query_batch_size`: 500
- `n_queries`: 19
- `unlabeled_size`: 50_000

In [3]:
experiment_name = 'jasiek_1'
N = 5
n_queries = 19
query_batch_size = 500
settings = pp.PipelineSettings(n_queries, 50, 50, query_batch_size, 'confidence')
train_size = 50_000
ratio_labeled = 0.01
test_size = 5000
loss_fn = F.cross_entropy

### Maciek
- `N`: 5
- `init`: 500
- `query_batch_size`: 100
- `n_queries`: 45
- `unlabeled_size`: 50_000

In [ ]:
experiment_name = 'maciek_1'
N = 5
n_queries = 45
query_batch_size = 100
settings = pp.PipelineSettings(n_queries, 50, 50, query_batch_size, 'confidence')
train_size = 50_000
ratio_labeled = 0.01
test_size = 5000
loss_fn = F.cross_entropy

### Szymon
- `N`: 5
- `init`: 100
- `query_batch_size`: 100
- `n_queries`: 49
- `unlabeled_size`: 50_000

In [ ]:
experiment_name = 'szymon_1'
N = 5
n_queries = 49
query_batch_size = 100
settings = pp.PipelineSettings(n_queries, 50, 50, query_batch_size, 'confidence')
train_size = 50_000
ratio_labeled = 0.002
test_size = 5000
loss_fn = F.cross_entropy

### Wiktor
- `N`: 3
- `init`: 500
- `query_batch_size`: 500
- `n_queries`: 39
- `unlabeled_size`: 50_000

In [6]:
experiment_name = 'wiktor_1'
N = 3
n_queries = 39
query_batch_size = 500
settings = pp.PipelineSettings(n_queries, 50, 50, query_batch_size, 'confidence')
train_size = 50_000
ratio_labeled = 0.01
test_size = 5000
loss_fn = F.cross_entropy

### Test (Ignore)

In [3]:
experiment_name = 'test'
N = 3
n_queries = 0
query_batch_size = 0
settings = pp.PipelineSettings(n_queries, 50, 50, query_batch_size, 'confidence')
train_size = 10_000
ratio_labeled = 1.0
test_size = 5_000
loss_fn = F.cross_entropy

### Wspólne (Oprócz Sebastiana)

In [4]:
dynamic_allocation = False
reset_on_every_trial = False

In [5]:
evaluation = em.Evaluation(262144)
accuracy_history = np.zeros(shape=(N, n_queries + 1))

if dynamic_allocation:
    dm.PCAMLL.reset(train_idx=[], test_idx=[], val_idx=[])
else:
    # by default all these arguments are None
    # if None PCAMLL initialises them to full arrays (i.e. all indices)
    dm.PCAMLL.reset() # allocates memory for all of the data (~4GB)

for i in range(N):
    print(f"trial: {i + 1}")
    
    model = models.PCAMClassifierSmall().to(device)
    learner = lm.ActiveLearner(model, device)
    optimizer = torch.optim.Adam(params=model.parameters())

    dataset = dm.ActiveDataset('pcam', train_size, test_size, ratio_labeled=ratio_labeled, balanced_split=False)
    if dynamic_allocation:
        if reset_on_every_trial: # reset on every trial
            dm.PCAMLL.reset(train_idx=[], test_idx=[], val_idx=[])
        dm.PCAMLL.append_idx('train', dataset.train_subset_idx)
        dm.PCAMLL.append_idx('val', dataset.test_subset_idx)

    pipeline = pp.Pipeline(dataset, learner, optimizer, loss_fn, settings)
    session, stats = pipeline.run(verbose=1, calculate_accuracy=True)
    evaluation.append(session)
    accuracy_history[i,:] = stats['accuracy_history']
print("Saving results")
evaluation.to_csv(f"{experiment_name}_evaluation.csv")
np.save(f'{experiment_name}_accuracy_history', accuracy_history)

trial: 1
trial: 2
trial: 3
Saving results


In [6]:
accuracy_history

array([[0.25626667],
       [0.25726666],
       [0.25186666]])

### Sebastian